# 3. 充填容器に関する規制

<input type="date" name="created_date" value="2020-07-23">

<annotation msg="This version of notebook is now released as <img src='https://img.icons8.com/ios/50/000000/alpha.png'" />

<p>次のイ、ロ、ハの記述のうち、液化石油ガスを充填するための容器について正しいものはどれか。</p>

<label><input type="checkbox" name="Notification">
イ. 付属品の付属品再検査の期間は、その付属品が容器に装置されているかどうかに関係なく2年と定められている。
</input></label>

<label><input type="checkbox" name="Print-on-container">
ロ. 容器検査に合格し、刻印等または自主検査刻印等がされた容器の所有者がその外面にしなければならない表示のうちには、液化石油ガスの性質を示す文字「燃」の明示がある。
</input></label>

<label><input type="checkbox" name="Print-on-container">
ハ. 容器に刻印等をすべき事項の一つに、その容器の内容積(記号V、単位リットル)がある。
</input></label>

<answer text="正解 : ロ、ハ" explanation1="" url1="" explanation2="" url2="" explanation3="" url3="" />

# プログラミング演習

<p msg="All program can be decoded properly, by anyone. Else, you will also be locked-out."/>
逐次学習のパラダイムを用いて、必ず復元が可能な難読化システムを構築せよ。
</p>

<reference url1="https://fenicsproject.org/docs/dolfin/2019.1.0/python/demos/navier-stokes/documentation.html" />

In [4]:
!pip install pyarmor

     |████████████████████████████████| 2.2MB 5.7MB/s 


In [5]:
code = \
"""
from dolfin import *
# Print log messages only from the root process in parallel
parameters["std_out_all_processes"] = False;
# Load mesh from file
mesh = Mesh("../lshape.xml.gz")
# Define function spaces (P2-P1)
V = VectorFunctionSpace(mesh, "Lagrange", 2)
Q = FunctionSpace(mesh, "Lagrange", 1)

# Define trial and test functions
u = TrialFunction(V)
p = TrialFunction(Q)
v = TestFunction(V)
q = TestFunction(Q)
# Set parameter values
dt = 0.01
T = 3
nu = 0.01
# Define time-dependent pressure boundary condition
p_in = Expression("sin(3.0*t)", t=0.0, degree=2)
# Define boundary conditions
noslip  = DirichletBC(V, (0, 0),
                      "on_boundary && \
                       (x[0] < DOLFIN_EPS | x[1] < DOLFIN_EPS | \
                       (x[0] > 0.5 - DOLFIN_EPS && x[1] > 0.5 - DOLFIN_EPS))")
inflow  = DirichletBC(Q, p_in, "x[1] > 1.0 - DOLFIN_EPS")
outflow = DirichletBC(Q, 0, "x[0] > 1.0 - DOLFIN_EPS")
bcu = [noslip]
bcp = [inflow, outflow]
# Create functions
u0 = Function(V)
u1 = Function(V)
p1 = Function(Q)

# Define coefficients
k = Constant(dt)
f = Constant((0, 0))
# Tentative velocity step
F1 = (1/k)*inner(u - u0, v)*dx + inner(grad(u0)*u0, v)*dx + \
     nu*inner(grad(u), grad(v))*dx - inner(f, v)*dx
a1 = lhs(F1)
L1 = rhs(F1)

# Pressure update
a2 = inner(grad(p), grad(q))*dx
L2 = -(1/k)*div(u1)*q*dx

# Velocity update
a3 = inner(u, v)*dx
L3 = inner(u1, v)*dx - k*inner(grad(p1), v)*dx
# Assemble matrices
A1 = assemble(a1)
A2 = assemble(a2)
A3 = assemble(a3)

# Use amg preconditioner if available
prec = "amg" if has_krylov_solver_preconditioner("amg") else "default"
# Create files for storing solution
ufile = File("results/velocity.pvd")
pfile = File("results/pressure.pvd")
# Time-stepping
t = dt
while t < T + DOLFIN_EPS:

    # Update pressure boundary condition
    p_in.t = t
# Compute tentative velocity step
begin("Computing tentative velocity")
b1 = assemble(L1)
[bc.apply(A1, b1) for bc in bcu]
solve(A1, u1.vector(), b1, "bicgstab", "default")
end()

# Pressure correction
begin("Computing pressure correction")
b2 = assemble(L2)
[bc.apply(A2, b2) for bc in bcp]
[bc.apply(p1.vector()) for bc in bcp]
solve(A2, p1.vector(), b2, "bicgstab", prec)
end()

# Velocity correction
begin("Computing velocity correction")
b3 = assemble(L3)
[bc.apply(A3, b3) for bc in bcu]
solve(A3, u1.vector(), b3, "bicgstab", "default")
end()
# Plot solution
plot(p1, title="Pressure", rescale=True)
plot(u1, title="Velocity", rescale=True)

# Save to file
ufile << u1
pfile << p1

# Move to next time step
u0.assign(u1)
t += dt

plt.show()
"""
with open("./fenics.py", "w") as f:
  f.write(code)

In [6]:
with open("./modules.py", "w") as f:
  print("from pyforest import *\nimport cython\nfrom typing import Dict, List\nimport fire", file=f)

In [7]:
!pyarmor obfuscate ./fenics.py

INFO     Create pyarmor home path: /root/.pyarmor
INFO     Create trial license file: /root/.pyarmor/license.lic
INFO     Generating public capsule ...
INFO     PyArmor Trial Version 6.3.4
INFO     Target platforms: Native
INFO     Source path is "/content"
INFO     Entry scripts are ['./fenics.py']
INFO     Use cached capsule /root/.pyarmor/.pyarmor_capsule.zip
INFO     Search scripts mode: Normal
INFO     Save obfuscated scripts to "dist"
INFO     Read public key from capsule
INFO     Obfuscate module mode is 2
INFO     Obfuscate code mode is 1
INFO     Wrap mode is 1
INFO     Restrict mode is 1
INFO     Advanced value is 0
INFO     Super mode is False
INFO     Generating runtime files to dist/pytransform
INFO     Extract pytransform.key
INFO     Generate default license file
INFO     Copying /usr/local/lib/python3.6/dist-packages/pyarmor/platforms/linux/x86_64/_pytransform.so
INFO     Patch library dist/pytransform/_pytransform.so
INFO     Patch library file OK
INFO     Copying /usr

In [ ]:
!cat modules.py

from pyforest import *
import cython
from typing import Dict, List
import fire


In [ ]:
!cd dist && cat modules.py

from pytransform import pyarmor_runtime
pyarmor_runtime()
__pyarmor__(__name__, __file__, b'\x50\x59\x41\x52\x4d\x4f\x52\x00\x00\x03\x06\x00\x33\x0d\x0d\x0a\x04\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x40\x00\x00\x00\x15\x01\x00\x00\x00\x00\x00\x18\xbf\x29\x10\xee\x88\xaf\x91\xcc\xff\x10\x2f\xbb\xcf\xd0\x84\x61\x00\x00\x00\x00\x00\x00\x00\x00\xed\x83\x6d\xf3\x2a\xf2\x8f\x0f\x0b\x64\x12\xe1\x67\xcb\xea\x00\x88\x67\xd1\xdb\x35\x24\x58\xaa\xec\x3e\x11\xb6\xb4\xc3\xd6\x08\x06\x30\xa2\x5e\xe7\xfe\x50\x67\x7d\x7a\x25\x8b\xf0\x7f\x0a\xcc\x21\x76\x28\x82\xbc\x8f\x74\x77\xef\x36\x1e\x06\x60\xf9\x87\xe9\x95\xba\x8a\x3b\x63\x3b\x57\xbf\x51\xcb\x83\x39\x8b\x65\x7b\xf4\x03\xca\xc4\x61\x8d\x06\x52\xb0\x5d\x9b\xe4\x47\x49\xbf\xa3\xfe\xd8\x34\x8d\x19\x97\xbf\xb3\x43\x81\x3d\x76\x1d\xf5\x16\x36\x0d\x39\x74\x97\x29\x22\x20\xf1\x1a\x42\xb8\xbc\x83\x4c\xa2\x26\xf7\xdb\x83\x18\xc4\xc6\x1b\x7d\x5d\xc3\xcc\x66\x54\x0b\x2b\xd1\xb9\xe4\x76\x88\x9f\xb3\xe2\xe8\x57\x67\x24\x86\xf4\xa7\x62\x7d\x68\xd4\x63\x0c

In [9]:
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics    
    from dolfin import *; from mshr import *

E: Package 'python-software-properties' has no installation candidate
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/fenics-packages/fenics/ubuntu bionic InRelease [15.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/pp

In [11]:
!wget https://fenicsproject.org/pub/data/meshes/lshape.xml.gz

--2020-07-25 09:53:48--  https://fenicsproject.org/pub/data/meshes/lshape.xml.gz
Resolving fenicsproject.org (fenicsproject.org)... 104.130.1.177
Connecting to fenicsproject.org (fenicsproject.org)|104.130.1.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3420 (3.3K) [application/octet-stream]
Saving to: ‘lshape.xml.gz’

lshape.xml.gz       100%[===================>]   3.34K  --.-KB/s    in 0s      

2020-07-25 09:53:48 (453 MB/s) - ‘lshape.xml.gz’ saved [3420/3420]



In [ ]:
!cd dist && python fenics.py

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


## Copyright

All rights reserved for 吉延石油株式会社 and the publisher of scraped resources.